# MVP Usage and Comparison

This notebook showcases the minimum amount of analysis needed to create a more effective
adversarial attack against a trained RL policy. The ARLIN library has additional
methods to gain more information that may be useful for an adversary, but the following
is the minimum needed to identify a terminal state with lower total reward.

### Environment

For this example, we will use the `LunarLander-v2` environment from `gymnasium`. In this
scenario, we are attempting to land a space vehicle with left, right, and vertical 
thrusters onto a landing pad without crashing into terrain obstacles.

### Goal

As an adversary, we are aiming to reduce the total reward gained by the agent indicating 
a failure to achieve the overall mission with high performance. To decrease the 
possibility of the attack being detected, we want to limit the number of adversarial 
attacks as much as possible while also ensuring that the attacks performed are not easily
detectable by a human observer or automated defense system.

In [ ]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import gymnasium as gym
import numpy as np
import logging
import warnings

import arlin.dataset.loaders as loaders
from arlin.dataset import XRLDataset
from arlin.dataset.collectors import SB3PPODataCollector, SB3PPODatapoint

from arlin.generation import generate_clusters, generate_embeddings
import arlin.analysis.visualization as viz
from arlin.analysis import ClusterAnalyzer, LatentAnalyzer
from arlin.samdp import SAMDP

logging.basicConfig(level=logging.INFO, force=True)
warnings.filterwarnings("ignore", category=UserWarning) 

np.random.seed(12345)

In [ ]:
def create_dataset():
    """Create an XRL Dataset from a trained model operating within an environment.
    """
    # Create environment
    env = gym.make("LunarLander-v2")
    
    # Load the SB3 model from Huggingface
    model = loaders.load_hf_sb_model(repo_id="sb3/ppo-LunarLander-v2",
                                     filename="ppo-LunarLander-v2.zip",
                                     algo_str="ppo")
    
    # Create the datapoint collector for SB3 PPO Datapoints with the model's policy
    collector = SB3PPODataCollector(datapoint_cls=SB3PPODatapoint,
                                    policy=model.policy)
    
    # Instantiate the XRL Dataset
    dataset = XRLDataset(env, collector=collector)
    
    # Fill the dataset with 50k datapoints and add in additional analysis datapoints
    dataset.fill(num_datapoints=50000)
    
    return dataset

In [ ]:
def get_embeddings(dataset: XRLDataset):
    """Generate latent space embeddings from the XRLDataset using T-SNE"""
    
    embeddings = generate_embeddings(dataset=dataset,
                                     activation_key="latent_actors",
                                     perplexity=225,
                                     n_train_iter=4000,
                                     output_dim=2,
                                     seed=12345)

    return embeddings

def get_clusters(dataset: XRLDataset):
    """Cluster the latent space embeddings using K-Means and MeanShift"""
    
    clusters, _, _, _ = generate_clusters(
        dataset,
        ["latent_actors", "critic_values"],
        ["latent_actors", "critic_values", "rewards"],
        ["rewards"],
        10,
        seed=1234
        )
    
    return clusters

In [ ]:
def graph_latent_analytics(embeddings: np.ndarray, 
                           clusters: np.ndarray, 
                           dataset: XRLDataset):
    """Graph visualizations of different latent space analytics over embeddings."""
    
    # Create a grapher to generate data used for analysis.
    grapher = LatentAnalyzer(embeddings, dataset)
    
    # Clusters
    cluster_data = grapher.clusters_graph_data(clusters)
    # Episode progression
    ep_prog_data = grapher.episode_prog_graph_data()
    # Greedy action confidence
    conf_data = grapher.confidence_data()
    
    base_path = os.path.join(".", "outputs", "mvp", "latent_analytics")
    for data in [(cluster_data, "clusters.png"),
                 (ep_prog_data, "episode_progression.png"),
                 (conf_data, "confidence.png")
                 ]:
        path = os.path.join(base_path, data[1])
        
        # Graph an individual data graph
        viz.graph_individual_data(path, data[0])
    
    # Graph multiple analytics as subplots in one plot
    combined_path = os.path.join(base_path, 'combined_analytics.png')
    viz.graph_multiple_data(file_path=combined_path,
                                           figure_title='Latent Analytics', 
                                           graph_datas=[conf_data, 
                                                        cluster_data, 
                                                        ep_prog_data])

In [ ]:
def graph_cluster_analytics(dataset, clusters):
    """Graph analytics for each cluster"""
    
    # Create grapher to graph cluster analytics
    grapher = ClusterAnalyzer(dataset, clusters)
    
    grapher.cluster_state_analysis(19,
                                   gym.make('LunarLander-v2'), 
                                   os.path.join(".", "outputs", "mvp", "cluster_state_analysis"))
    
    # Mean confidence per cluster
    cluster_conf = grapher.cluster_confidence()
    # Mean total reward per cluster
    cluster_rewards = grapher.cluster_rewards()
    # Mean value per cluster
    cluster_values = grapher.cluster_values()
    
    # Graph individual graphs per data
    base_path = os.path.join(".", "outputs", "mvp", 'cluster_analytics')
    for data in [[cluster_conf, 'cluster_confidence.png'], 
                 [cluster_rewards, 'cluster_rewards.png'],
                 [cluster_values, 'cluster_values.png']
                 ]:
        path = os.path.join(base_path, data[1])
        viz.graph_individual_data(path, data[0])
    
    # Graph multiple subplots in one plot
    combined_path = os.path.join(base_path, 'combined_analytics.png')
    viz.graph_multiple_data(file_path=combined_path, 
                                           figure_title='Cluster Analytics', 
                                           graph_datas=[cluster_rewards, 
                                                        cluster_conf,
                                                        cluster_values])

In [ ]:
def samdp(clusters: np.ndarray,
          dataset: XRLDataset):
    """Generate a semi-aggregated Markov decision process."""
    
    # Create the SAMDP
    samdp = SAMDP(clusters, dataset)
    
    base_path = os.path.join(".", "outputs", "mvp", 'samdp')
    
    # Simplified graph with all possible conenctions (regardless of action taken)
    _ = samdp.save_simplified_graph(f'{base_path}/samdp_simplified.png')
    
    # Show all connections to terminal nodes
    samdp_term = os.path.join(base_path, f"samdp_terminal_paths.png")
    samdp.save_terminal_paths(samdp_term, best_path=True)

In [ ]:
dataset = create_dataset()
embeddings = get_embeddings(dataset)
clusters = get_clusters(dataset)

graph_latent_analytics(embeddings, clusters, dataset)
graph_cluster_analytics(dataset, clusters)
samdp(clusters, dataset)